In [1]:
import re
from bs4 import BeautifulSoup 

In [2]:
def review_to_wordlist(review):
    review_text = BeautifulSoup(review).get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    return(words)

In [3]:
import pandas as pd

In [4]:
train = pd.read_csv('Individual assaignment data mining/Gegeven dataset/labeledTrainData.tsv', header=0,
                delimiter="\t", quoting=3)
test = pd.read_csv('Individual assaignment data mining/Gegeven dataset/testData.tsv', header=0, delimiter="\t",
               quoting=3 )

In [5]:
y_train = train['sentiment']
X_train = train['review']

In [6]:
traindata = []
for i in range(0,len(X_train)):
    traindata.append(" ".join(review_to_wordlist(X_train[i])))
testdata = []
for i in range(0,len(X_train)):
    testdata.append(" ".join(review_to_wordlist(X_train[i])))

/Users/gladwellacademy/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/gladwellacademy/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf(X_train = None, X_test = None, ngram_range = (1 ,2)):
    if ngram_range:
        ngram_range = ngram_range
    else:
        ngram_range = (1 ,2)
        
    tfv = TfidfVectorizer(min_df=3,  max_features=None,
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=ngram_range, use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')
    
    if X_train:
        Xfit = tfv.fit(X_train) 
        train = tfv.transform(X_train)
    else: train = 'no training data provided'
    if X_test:
        Xtestfit = tfv.fit(X_test)
        test = tfv.transform(X_test)
    else: test = 'no testing data provided'
    return(train, test)

In [8]:
X_train, X_test = tfidf(X_train = traindata, X_test = testdata)

In [9]:
from sklearn.model_selection import train_test_split

rvw_train, rvw_test, label_train, label_test = \
train_test_split(X_train, train['sentiment'], test_size=0.2)

###  Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV

/Users/gladwellacademy/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/gladwellacademy/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [11]:
param_grid = [{'dual': [False], 'penalty': ['l1'], 'C': [1, 10]}, 
              {'dual': [True], 'penalty':['l2'], 'C': [1, 10 ,100]}]

model_LR = GridSearchCV(LogisticRegression(penalty = 'L2', dual = True, random_state = 0), 
                        param_grid, scoring = 'roc_auc', cv = 20) 
                        
model_LR.fit(rvw_train,label_train)

GridSearchCV(cv=20, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='L2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'dual': [False], 'penalty': ['l1'], 'C': [1, 10]}, {'dual': [True], 'penalty': ['l2'], 'C': [1, 10, 100]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [12]:
from sklearn.metrics import classification_report
predictionsLR = model_LR.predict(rvw_test)
print (classification_report(predictionsLR, label_test))

             precision    recall  f1-score   support

          0       0.88      0.90      0.89      2410
          1       0.90      0.88      0.89      2590

avg / total       0.89      0.89      0.89      5000



In [13]:
model_LR_best = model_LR.best_estimator_
model_LR_best

LogisticRegression(C=10, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Multinominal Naive Bayes

In [14]:
from sklearn.naive_bayes import MultinomialNB

In [15]:
model_MNB = MultinomialNB()
model_MNB.fit(rvw_train,label_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
predictionsMNB = model_MNB.predict(rvw_test)
print (classification_report(predictionsMNB, label_test))

             precision    recall  f1-score   support

          0       0.88      0.87      0.87      2496
          1       0.87      0.88      0.87      2504

avg / total       0.87      0.87      0.87      5000



###  Stochastic Gradient Descent

In [17]:
from sklearn.linear_model import SGDClassifier

In [18]:
sgd_params = {'alpha': [0.00006, 0.00007, 0.00008, 0.0001, 0.0005]} 

model_SGD = GridSearchCV(SGDClassifier(random_state = 0, shuffle = True, loss = 'modified_huber'), 
                        sgd_params, scoring = 'roc_auc', cv = 20)

model_SGD.fit(rvw_train,label_train) 

predictionsSGD = model_SGD.predict(rvw_test)
print (classification_report(predictionsSGD, label_test))

GridSearchCV(cv=20, error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [6e-05, 7e-05, 8e-05, 0.0001, 0.0005]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [19]:
predictionsSGD = model_SGD.predict(rvw_test)
print (classification_report(predictionsSGD, label_test))

             precision    recall  f1-score   support

          0       0.88      0.90      0.89      2425
          1       0.90      0.89      0.89      2575

avg / total       0.89      0.89      0.89      5000



In [20]:
model_SGD_best = model_SGD.best_estimator_
model_SGD_best

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False)

### Random Forrest 

In [21]:
from sklearn.ensemble import RandomForestClassifier

#use a large GridSearch over all parameters
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

#Run GridSearch and fitting model
model_RFC = GridSearchCV(RandomForestClassifier(n_estimators = 100), param_grid=param_grid)

model_RFC.fit(rvw_train,label_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, None], 'max_features': [1, 3, 10], 'min_samples_split': [2, 3, 10], 'min_samples_leaf': [1, 3, 10], 'bootstrap': [True, False], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [22]:
predictionsRFC = model_RFC.predict(rvw_test)
print (classification_report(predictionsRFC, label_test))

             precision    recall  f1-score   support

          0       0.84      0.86      0.85      2412
          1       0.87      0.85      0.86      2588

avg / total       0.85      0.85      0.85      5000



In [23]:
model_RFC_best = model_RFC.best_estimator_
model_RFC_best

RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=None, max_features=10,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [24]:
def classifier(X_train, y, clf, predict = 'evaluate', X_test = None):
    if predict == 'evaluate':
        name = clf.__class__.__name__
        rvw_train, rvw_test, label_train, label_test = \
        train_test_split(X_train, y, test_size=0.2) 
        clf.fit(rvw_train,label_train)
        predictionsCLF = clf.predict(rvw_test)
        print(name,"\n", classification_report(predictionsCLF, label_test))
    elif predict == 'submit':
        clf.fit(X,y)
        return clf.predict(X_test)

In [25]:
#Models based on best_estimator
classifier(X_train, y_train, model_LR_best)
classifier(X_train, y_train, model_MNB)
classifier(X_train, y_train, model_RFC_best)
classifier(X_train, y_train, model_SGD_best)

LogisticRegression 
              precision    recall  f1-score   support

          0       0.89      0.90      0.90      2439
          1       0.91      0.89      0.90      2561

avg / total       0.90      0.90      0.90      5000

MultinomialNB 
              precision    recall  f1-score   support

          0       0.87      0.88      0.88      2522
          1       0.88      0.87      0.87      2478

avg / total       0.88      0.88      0.88      5000

RandomForestClassifier 
              precision    recall  f1-score   support

          0       0.84      0.86      0.85      2437
          1       0.86      0.85      0.86      2563

avg / total       0.85      0.85      0.85      5000

SGDClassifier 
              precision    recall  f1-score   support

          0       0.90      0.90      0.90      2522
          1       0.90      0.90      0.90      2478

avg / total       0.90      0.90      0.90      5000



In [26]:
from sklearn.ensemble import VotingClassifier

weights = {'weights': [[1, 1, 1, 1],[2, 2, 2, 1],[2, 2, 1, 2],[2, 1, 2, 2],[3, 3, 2, 1]]} 

model_Voting1 = GridSearchCV(VotingClassifier(estimators=[('SGD', model_SGD_best), ('LR', model_LR_best),
                                    ('MNB', model_MNB),('RFC', model_RFC_best)],
                                    voting='soft'), weights)

# Niet runnen

In [27]:
classifier(X_train, y_train, model_Voting1)

GridSearchCV 
              precision    recall  f1-score   support

          0       0.89      0.91      0.90      2422
          1       0.91      0.89      0.90      2578

avg / total       0.90      0.90      0.90      5000



In [28]:
model_Voting1.best_estimator_

VotingClassifier(estimators=[('SGD', SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
   ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))],
         n_jobs=1, voting='soft', weights=[3, 3, 2, 1])

In [29]:
from sklearn.ensemble import VotingClassifier
weights = [[x, y, z] for x in range(4) for y in range(4) for z in range(4)]
del [weights[weights.index([0,0,0])], weights[weights.index([1,1,1])], weights[weights.index([2,2,2])]]

voting_weights = {'weights': weights}

model_Voting2 = GridSearchCV(VotingClassifier(estimators=[('SGD', model_SGD), ('LR', model_LR),
                                    ('MNB', model_MNB)],
                                    voting='soft'), voting_weights)

In [30]:
classifier(X_train, y_train, model_Voting2)

GridSearchCV 
              precision    recall  f1-score   support

          0       0.90      0.90      0.90      2484
          1       0.90      0.91      0.90      2516

avg / total       0.90      0.90      0.90      5000



In [31]:
model_Voting2.best_estimator_

VotingClassifier(estimators=[('SGD', GridSearchCV(cv=20, error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=5, n_jobs=1,
       penalty='l2',...scoring='roc_auc', verbose=0)), ('MNB', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         n_jobs=1, voting='soft', weights=[3, 1, 2])

In [32]:
X_train_3, X_test_3 = tfidf(X_train = traindata, X_test = testdata, ngram_range = (0, 3))

rvw_train_3, rvw_test_3, label_train_3, label_test_3 = \
train_test_split(X_train_3, train['sentiment'], test_size=0.2)

In [33]:
X_train_4, X_test_4 = tfidf(X_train = traindata, X_test = testdata, ngram_range = (0, 4))

rvw_train_4, rvw_test_4, label_train_4, label_test_4 = \
train_test_split(X_train_4, train['sentiment'], test_size=0.2)

In [34]:
model_SGD = GridSearchCV(SGDClassifier(random_state = 0, shuffle = True, loss = 'modified_huber'), 
                        sgd_params, scoring = 'roc_auc', cv = 20)

#Trigram
model_SGD.fit(rvw_train_3,label_train_3) 

predictionsSGD = model_SGD.predict(rvw_test_3)
print (classification_report(predictionsSGD, label_test_3))

#Four-gram
model_SGD.fit(rvw_train_4,label_train_4) 

predictionsSGD = model_SGD.predict(rvw_test_4)
print (classification_report(predictionsSGD, label_test_4))

             precision    recall  f1-score   support

          0       0.89      0.91      0.90      2413
          1       0.91      0.90      0.91      2587

avg / total       0.90      0.90      0.90      5000

             precision    recall  f1-score   support

          0       0.90      0.90      0.90      2509
          1       0.90      0.90      0.90      2491

avg / total       0.90      0.90      0.90      5000



In [37]:
param_grid = [{'dual': [False], 'penalty': ['l1'], 'C': [1, 10]}, 
              {'dual': [True], 'penalty':['l2'], 'C': [1, 10 ,100]}]

model_LR = GridSearchCV(LogisticRegression(penalty = 'L2', dual = True, random_state = 0), 
                        param_grid, scoring = 'roc_auc', cv = 20) 
                        
#Trigram
model_LR.fit(rvw_train_3,label_train_3)

predictionsLR = model_LR.predict(rvw_test_3)
print (classification_report(predictionsLR, label_test_3))

#Four-gram
model_LR.fit(rvw_train_4,label_train_4)

predictionsLR = model_LR.predict(rvw_test_4)
print (classification_report(predictionsLR, label_test_4))

             precision    recall  f1-score   support

          0       0.89      0.91      0.90      2396
          1       0.92      0.90      0.91      2604

avg / total       0.90      0.90      0.90      5000

             precision    recall  f1-score   support

          0       0.90      0.91      0.90      2484
          1       0.91      0.90      0.90      2516

avg / total       0.90      0.90      0.90      5000

